In [1]:
import torch
from model_2 import Model
import matplotlib.pyplot as plt
import torchvision.transforms as T 
from PIL import Image
device =  'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model = Model()

path_train = '../data/train_data.pkl'
path_val = '../data/val_data.pkl'

noisy_imgs_1, noisy_imgs_2 = torch.load(path_train)
noisy_imgs_1 = noisy_imgs_1[0:1000].to(device)
noisy_imgs_2 = noisy_imgs_2[0:1000].to(device)

noisy_imgs , clean_imgs = torch.load(path_val)
noisy_imgs = noisy_imgs[0:1000].to(device)
clean_imgs = clean_imgs[0:1000].to(device)


cuda


In [2]:
print(torch.empty(1))

tensor([0.])


In [3]:
model.optimizer.lr = 1e-3
model.optimizer.set_momentum(0.1)
model.train(noisy_imgs_1, noisy_imgs_2, 200)

out = model.predict(noisy_imgs)

100%|████████████████████████████████████████████████████████████████████████████████| 200/200 [01:28<00:00,  2.26it/s]


RuntimeError: CUDA out of memory. Tried to allocate 862.00 MiB (GPU 0; 4.00 GiB total capacity; 2.02 GiB already allocated; 0 bytes free; 2.55 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import matplotlib.pyplot as plt
plt.plot(model.loss_train.cpu())

In [ ]:
transform = T.ToPILImage()
plt.figure(figsize = [10, 5])
plt.subplot(1, 3,1)
plt.imshow(transform(noisy_imgs[20]));
plt.subplot(1, 3,2)
plt.imshow(transform(out[20]/256));
plt.subplot(1, 3,3)
plt.imshow(transform(clean_imgs[20]));